In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#ignore comment

In [3]:
import pandas as pd
import numpy as np

import pyarrow.parquet as pq

import time
import datetime as dt


from pathlib import Path

#### Adjust pandas Outcput

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [5]:
%%time
canary_p = '../../data/CANARY/CANARY/PERSON_SUMMARY'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pq.read_table(canary_p).to_pandas()
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 27.6 s, sys: 9.18 s, total: 36.7 s
Wall time: 19.4 s


In [6]:
pmpm.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2570122               95      40392        0             0                  1       0          1                    0             0          0              1          0           0       1                 0            0                  0                  1                     0        1                   0              1                        0                  0            0                0         12                      0            0                0            1                  0                 1               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2570122               94      19837        0             0                  0       0          1                    0             1          0              0          0           0       1                 0            0                  0                  0                     0        1                   0              0                        0                  0            0                0          7                      0            0                0            1                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
2  2570268               93       4922        0             1                  1       0          1                    0             1          1              1          0           0       0                 0            0                  0                  1                     0        1                   0              0                        0                  0            0                0         11                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             0         0               0            0                   0                      0                      0           0                0              0               0   
3  2570268             

#### Data Processing and Filtering
1. For Biometric data
2. For PMPM Data
3. For Canary data

#### 1) Biometric data 
1. Pick Category of data
1. Standarize Dates: MYR, Datetime Date format
2. Average Values: In the same Month Year

In [7]:
biometrics.columns

Index(['PERS_ID', 'EVENT_DATE', 'DATE_TYPE', 'BIOMETRIC_CATEGORY', 'EVENT_CODE', 'CODING_STANDARD', 'SOURCE', 'RESULT_VALUE'], dtype='object')

#### An inspection of categories registered in biometrics data

In [8]:
%%time
biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)

CPU times: user 19.9 s, sys: 4.93 s, total: 24.8 s
Wall time: 24.8 s


BIOMETRIC_CATEGORY
A1C                             790853
BMI                           10034183
BNP                              42552
Blood Pressure (Diastolic)     4902004
Blood Pressure (Systolic)      5238991
Cholesterol (LDL)              1305880
DCSI-EGFR                       628758
DCSI-MICROALBUMIN                81297
DCSI-SERUM                     2734084
Ferritin                        219613
Height                        11808045
Hematocrit                     2574412
Hemoglobin                     2919909
Iron                            197878
NT_proBNP                         9258
PSA                             310283
Potassium                      2666981
Troponin                         94030
Weight                        14000267
eGFR                            907656
dtype: int64

#### Above we see categories we would like to pick for outcomes (Pick BMI for example)
#### Pick a category as a base for outcome creation

In [9]:
%%time
filt = biometrics.BIOMETRIC_CATEGORY == 'BMI'
biometrics = biometrics[filt].reset_index(drop = True)

#make ids integers, and result values floats
biometrics['PERS_ID'] = biometrics['PERS_ID'].astype(int)
biometrics.RESULT_VALUE = biometrics.RESULT_VALUE.astype(float)

#make dates datetime date
biometrics.EVENT_DATE = pd.to_datetime(biometrics.EVENT_DATE).dt.date

CPU times: user 13.9 s, sys: 1.98 s, total: 15.9 s
Wall time: 15.9 s


#### Make Biometrics EVENT_DATE_MYR and all EVENT_DATE dates to start from day 1

In [10]:
def make_days1( row ):
    """
    takes a datetime.date
    object and makes the day 1
    """
    year = row.year
    month = row.month
    
    return dt.date(year = year, month = month, day = 1)

def make_date_to_MYR( row ):
    """
    takes a date object
    and returns a 6 digit integer
    year-month, ex:199302
    
    """
    
    new_row = row.year*100 +row.month
    
    return new_row

#### Apply Date modifications to Biometrics data

In [11]:
%%time
biometrics.EVENT_DATE = biometrics.EVENT_DATE.apply(make_days1)
biometrics['EVENT_DATE_MYR'] = biometrics.EVENT_DATE.apply(make_date_to_MYR)

CPU times: user 12.2 s, sys: 712 ms, total: 12.9 s
Wall time: 12.9 s


#### Compute the Mean of biometric values occuring the same date and keep this as the new value, drop the duplicates

In [12]:
%%time
#compute the mean biometric value per month for each patient
bio_mean = biometrics.groupby(by = ['PERS_ID', 'EVENT_DATE_MYR']).RESULT_VALUE.mean()

#drop the duplicate ids-dates tuples and the biometrics results values
biometrics.drop_duplicates(subset = ['PERS_ID', 'EVENT_DATE_MYR'], inplace = True)
biometrics.drop(labels = 'RESULT_VALUE', axis = 1, inplace = True)

#merge the mean result values in the initial biometrics dataset
biometrics = biometrics.merge(bio_mean, on = ['PERS_ID', 'EVENT_DATE_MYR'])

#drop unecessary columns from biometrics
biometrics.drop(labels = ['DATE_TYPE', 'CODING_STANDARD', 'SOURCE', 'EVENT_CODE'], inplace = True, axis = 1)

CPU times: user 4.86 s, sys: 1.84 s, total: 6.69 s
Wall time: 6.69 s


In [13]:
biometrics.head(2)

PERS_ID  EVENT_DATE BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE
0  2570719  2018-12-01                BMI          201812     26.473165
1  2570719  2020-12-01                BMI          202012     25.840080

#### Drop Ids in Biometrics that do not have at least two measurements on record in different dates 

In [14]:
%%time
number_of_measurements = biometrics.groupby('PERS_ID').apply(len)
ids_more_than2 = number_of_measurements[number_of_measurements > 1].index

biometrics = biometrics[biometrics.PERS_ID.isin(ids_more_than2)]

CPU times: user 1.77 s, sys: 212 ms, total: 1.99 s
Wall time: 1.99 s


In [15]:
biometrics.PERS_ID.unique().shape, ids_more_than2.shape, number_of_measurements.shape

((261602,), (261602,), (318584,))

#### PMPM_PROCESSING
Make ids integers, make gender a binary variable (0,1)

In [16]:
%%time
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)
pmpm.MBR_GNDR = pmpm.MBR_GNDR.replace(['F', 'M'], [0,1])

CPU times: user 2.67 s, sys: 373 ms, total: 3.04 s
Wall time: 3.04 s


#### Continuous enrollement Filter
#### Function to check on the continous enrollement of patients in PMPM dataset

In [17]:
def is_consecutive(date1, date2):
    """
    Checks if two dates in
    YYYYMM format are consecutive
    for example 
    date1 = 201809, date2 = 201810--> True
    date1 = 201812, date2 = 201901 -->True
    date1 = 201708, date2 = 201710 --> False
    
    """
    
    m1, y1 = date1%100, date1//100
    m2, y2 = date2%100, date2//100
    
    if m2%12 == (m1%12 + 1):
        if (y1 == y2) or (y2 == y1+1):
            return True
        
    if (m1 == 11) and (m2 == 12):
        if (y1 == y2) or (y2 == y1+1):
            return True
        
    
    return False
    
def enrollement_check( data, time_in_months):
    """
    Finds All Patients with Continuous Enrollement
    data in a Nx2 matrix, N is the size of PMPM
    columns: 1:ids, 2:Myr, 
    
    data: Nx2 matrix first column is the patients Ids
    second column is the time in months that a patient
    needs to be enrolled consecutively
    """
    
    tm = time_in_months
    
    #first patient id to check
    current_id = data[0,0]
    counter = 1
    
    #corresponding year month date
    current_date = data[0,1]
    start_date = current_date
    N = data.shape[0]
    enhanced_data = []
    
    #for all dates and ids in dataset
    for i in range(1,N):
        idi = data[i, 0]
        date_i = data[i, 1]
        
        if idi == current_id: #we still checking a specific patient
            if is_consecutive(current_date, date_i):
                counter += 1
            else:
                if counter >= tm:
                    enhanced_data.append([current_id, start_date, current_date, counter])
                
                counter = 1
                start_date = date_i
        
        else: #change id
            if counter >= tm: #if previous id was satisfying enrollment criteria
                enhanced_data.append([current_id, start_date, current_date, counter])
            
            #reseting start date
            start_date = date_i
            
            #reseting counter
            counter = 1
        
        #prepare date and id for next iteration
        current_date = date_i
        current_id = idi
        
    return enhanced_data

#entry point for enrollemnet filter returns a panda array
def customize_data(data, time_in_months):
    """
    data: Nx2 matrix where first column is the ids and second 
    column the corresponding yearmonth
    """
    dat = enrollement_check(data, time_in_months)
    
    columns = ['PERS_ID', 'FIRST_DATE', 'LAST_DATE', 'MONTH_COUNTER']
    
    return pd.DataFrame(dat, columns = columns)
    
    
    
    

#### Apply the enrollement criteria algorithm  in PMPM data

In [18]:
%%time
# sort pmpm by ids and dates for the enrollment algorithm to work
pmpm.sort_values(by = ['PERS_ID', 'MYR'], inplace = True)
pmpm.reset_index(drop = True, inplace = True)
pmpm_test = pmpm[['PERS_ID', 'MYR']].values

#### find ids with contiuous enrollement up to 12 months
months_of_enrollement = 12
dat = customize_data(pmpm_test, months_of_enrollement)

CPU times: user 19.4 s, sys: 3.62 s, total: 23 s
Wall time: 23 s


In [19]:
dat.PERS_ID.unique().shape, pmpm.PERS_ID.unique().shape

((228881,), (328215,))

#### Keep only ids that satisfy the enrollement criteria

In [20]:
%%time
unique_ids = dat.PERS_ID.unique()
pmpm = pmpm[pmpm.PERS_ID.isin(unique_ids)]
print(pmpm.PERS_ID.unique().shape)

(228881,)
CPU times: user 1.3 s, sys: 860 ms, total: 2.16 s
Wall time: 2.16 s


In [21]:
pmpm.shape, biometrics.shape, pmpm.PERS_ID.unique().shape, biometrics.PERS_ID.unique().shape

((7262669, 74), (3237619, 5), (228881,), (261602,))

#### At this Point we have a Processed  Biometrics data file and a fitered PMPM file by Enrollement Criteria
We can merge the two of them on ids and dates

In [22]:
%%time
merged = pmpm.merge(biometrics, left_on = ['PERS_ID', 'MYR'], right_on = ['PERS_ID', 'EVENT_DATE_MYR'])
merged.sort_values(by = ['PERS_ID', 'MYR'], inplace = True, ignore_index = True)

CPU times: user 2.98 s, sys: 661 µs, total: 2.98 s
Wall time: 2.98 s


In [23]:
merged.head(3)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2561296               38        158        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2561296               39        173        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
2  2562798              102       1376        0             0                  0       0          0                    0             0          0              0          0           0       0                 1            0                  0                  0                     0        0                   0              0                        0                  0            0                1          3                      0            0                0            1                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PRO

<b>If we try to enforce both pmpm and biometric continuous measurements (for example bmi) <br>
we end up with 273 IDS, so we will go by considering measurements <br>
close to refernce dates (that we will compute later)

In [24]:
dat2 = customize_data(merged[['PERS_ID', 'MYR']].values, 12)
dat2.PERS_ID.unique().shape

(723,)

In [25]:
merged.shape

(1137729, 78)

#### Canary Processing
1. Technical Process (format columns in appropriate form, ids dates etc)
2. Filtering, keep only ids for which we have data


In [26]:
#drop extra columns
canary.drop(labels = ['MBR_ID', 'GROUPNUMBER', 'GROUPNAME', 'LOB', 'FILE_NAME', 'FILE_DATE', 
                      'LAST_COMPLETED_MILESTONE'], axis = 1, inplace = True)

canary.PERS_ID = canary.PERS_ID.astype(int)

#### Fix dates format make MYR format

In [27]:
canary.REGISTER_DATE = canary.REGISTER_DATE.apply(make_days1)
canary.LAST_ACTIVITY_DATE = canary.LAST_ACTIVITY_DATE.apply(make_days1)
canary['REGISTER_DATE_MYR'] = canary.REGISTER_DATE.apply(make_date_to_MYR)
canary['LAST_ACTIVITY_MYR'] = canary.LAST_ACTIVITY_DATE.apply(make_date_to_MYR)


#### Filter canary data with ids only in merged pmpm - biometrics

In [28]:
canary = canary[canary.PERS_ID.isin(merged.PERS_ID.unique())]
canary.shape

(1064, 6)

#### For non Treated the Outcomes will be taken Sequentially.
#### For Treated the Outcomes will be taken close to registration and end date

In [29]:
merged_ids = merged.PERS_ID.unique()
canary_ids = canary.PERS_ID.unique()

control_ids = np.setdiff1d(merged_ids, canary_ids)
print(len(control_ids), len(merged_ids), len(canary_ids))

161278 162342 1064


#### Separate Control and Treated Patients

In [30]:
filt_cont = merged.PERS_ID.isin(control_ids)
controls = merged[filt_cont].reset_index(drop = True)
treated  = merged[~filt_cont].reset_index(drop = True)

In [31]:
controls.PERS_ID.unique().shape, treated.PERS_ID.unique().shape

((161278,), (1064,))

#### Calculate the sequential outcomes of controls and at the end drop the last month of each Id (because there is no further data for the outcome)
1. OUTCOME: 1 if positive 0 if negative(measures if the measuremnt under consideration increased or decreased between the pre treatment and post treatment dates)
2. OUTCOME2: Measures the increase percentage of the value under consideration

In [32]:
res_value = controls.RESULT_VALUE.values
outcomes = ((res_value[1:]-res_value[0:-1]) < 0).astype(int)
outcome2 = ((res_value[1:]-res_value[0:-1])/res_value[0:-1])

controls = controls.iloc[0:-1]
controls['OUTCOME'] = outcomes
controls['OUTCOME2'] = outcome2


controls = controls.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)
controls.shape[0] 

1126574

In [33]:
controls.head(5)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2561296               38        158        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2561296               39        173        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
2  2562798              102       1376        0             0                  0       0          0                    0             0          0              0          0           0       0                 1            0                  0                  0                     0        0                   0              0                        0                  0            0                1          3                      0            0                0            1                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               1             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
3  2562798             

In [34]:
def drop_last(group):
    return group.iloc[0:-1]

In [35]:
%%time
final_dates = controls.groupby('PERS_ID')['MYR'].apply(drop_last).droplevel(1).reset_index()
controls_final = pd.merge(controls, final_dates, on = ['PERS_ID', 'MYR'], how = 'inner')
controls_final['TREATED'] = 0

CPU times: user 39.1 s, sys: 152 ms, total: 39.2 s
Wall time: 39.2 s


#### Keep only Measurements that were taken at least 120 days apart  and at most 180 days apart(these ar parameters that can be modified)

In [36]:
days_at_least = 120
days_at_most = 180
date_diff = pd.Series(controls_final.EVENT_DATE.iloc[1:].values - controls_final.EVENT_DATE.iloc[0: -1].values).dt.days
time_filt = (date_diff >= days_at_least) & (date_diff <= days_at_most)

#remove outcomes that do not satisfy the time filter
controls_final = controls_final.iloc[0:-1][time_filt].reset_index(drop = True)


In [37]:
time_filt.sum(), date_diff.shape

(112540, (965295,))

In [38]:
controls_final.shape, controls.shape

((112540, 81), (1126574, 80))

#### Now find the outcomes for treated data

In [39]:
treated = treated.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)
treated2 = pd.merge(treated, canary, on = 'PERS_ID')
treated2.shape

#Find days between the measurements and the registration and end treatment dates
treated2['REF_START']  = np.abs(treated2.EVENT_DATE - treated2.REGISTER_DATE)
treated2['REF_END']  = np.abs(treated2.EVENT_DATE - treated2.LAST_ACTIVITY_DATE)

In [40]:
treated2.head(2)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  \
0  2602482               80       1232        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          4                      0            0                0            1                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   
1  2602482               80        244        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          4                      0            0                0            1                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0   

   CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR     MYR  EVENT_DATE BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE  N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE  REGISTER_DATE_MYR  LAST_ACTIVITY_MYR REF_START  REF_END  
0            0                   0        0                 0                       0               0                0          0           0                   0                 0         0  201803  2018-03-01                BMI          201803     29.094867                      1    2019-08-01         2019-08-01             201908             201908  518 days 518 days  
1            0                   0        0                 0                       0               0                0          0           0                   0                 0         0  201808  2018-08-01                BMI          201808     27.790791           

In [41]:
def pick_covariates( group ):
    """
    choses rows closer 
    to registration and last activity date
    
    """
    indx_start = np.argmin(group['REF_START'].values)
    indx_end   = np.argmin(group['REF_END'].values)
    N = len(group) - 1
    if indx_start == indx_end:
        if indx_start != N:
            indx_end = N
        else:
            indx_start = 0
    measure_start = group.iloc[indx_start].EVENT_DATE
    measure_end   = group.iloc[indx_end].EVENT_DAE
            
    data = group.iloc[[indx_start, indx_end]]    
    data['FF'] = 0
    data.iloc[1,-1] = 1
    data['OUTCOME'] = int((data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])<0)
    data['OUTCOME2']  = (data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])/data['RESULT_VALUE'].iloc[0]
    return data
        
### A SECOND MOST SEVERE FILTERING ON ACQUIRING TREATED OUTCOMES     
def pick_covariates2(group, days_pre, days_post):
    """
    more strict criteria on acquiring covariates and outcomes
    """
    
    #get only measuremnts before register date
    start_dates = group.EVENT_DATE <= group.REGISTER_DATE
    
    #get only measurements after last activity date
    end_dates = group.EVENT_DATE >= group.LAST_ACTIVITY_DATE
    
    group_st = group[start_dates]
    
    #require start data at most 120 days before register date
    group_st = group_st[group_st['REF_START'].dt.days <= days_pre]
    
    if not group_st.empty: #if no dates find return NULL
        group_st = group_st.sort_values(by = 'REF_START').iloc[0]
    else:
        return
    
    #now for the end dates
    group_en = group[end_dates]
    
    #require end data at least 120 days after last activity date and at most 180 days
    #differentiate the cases where register and end date are identical or not
    if group.REGISTER_DATE.iloc[0] != group.LAST_ACTIVITY_DATE.iloc[0]: #if the two dates are different then allow at most 120 after end date for measuremnt
        group_en = group_en[(group_en['REF_END'].dt.days <= days_post) & (group_en['REF_END'].dt.days >= 0)  ]
    else:
        #if dates are the same require at least 120 days diffence
        group_en = group_en[(group_en['REF_END'].dt.days <= days_post) & (group_en['REF_END'].dt.days >= days_pre)  ]
    #print(group_en)
    if not group_en.empty:
       # print('HERE')
        group_en = group_en.sort_values(by = 'REF_END').iloc[0]
    else:
       # print(group_en)
        return
        
    data = pd.DataFrame((group_st.values, group_en.values), columns = group_st.index)
    data['FF'] = 0
    data.iloc[1,-1] = 1
    data['OUTCOME'] = int((data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])<0)
    data['OUTCOME2']  = (data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])/data['RESULT_VALUE'].iloc[0]
    
    return data
    

#### Construct treated data outcomes

In [42]:
%%time
treated3 = treated2.groupby(by = 'PERS_ID').apply(lambda d: pick_covariates2(d,120,180))
treated3.index = range(len(treated3))
treated_help = treated3.copy()
treated3 = treated3[treated3.FF == 0].reset_index(drop = True)
treated3['TREATED'] = 1

CPU times: user 8.37 s, sys: 811 µs, total: 8.37 s
Wall time: 8.37 s


In [43]:
common_columns = np.intersect1d(controls_final.columns, treated3.columns)
common_columns[0:10]

array(['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'BIOMETRIC_CATEGORY', 'CC_ADHD',
       'CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_AMI', 'CC_ANEMIA',
       'CC_ANXIETY_DISORDER', 'CC_ARTHRITIS'], dtype=object)

In [44]:
treated_final = treated3[common_columns]
treated_final.shape

(417, 81)

#### Concatenate controls and treated

In [45]:
df = pd.concat((treated_final, controls_final), axis = 0, ignore_index = True)
df.shape, treated_final.shape, controls_final.shape

((112957, 81), (417, 81), (112540, 81))

#### drop extra columns

In [46]:
to_drop = ['EVENT_DATE', 'EVENT_DATE_MYR', 'MYR']
df = df.drop(labels = to_drop, axis = 1)

In [47]:
df.head(3)

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0               81        414                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                1          2                      0            0                0            0                  1                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
1               82        101                BMI        0             0                  0       0          0                    1             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0   
2               78         72                BMI        0             0                  0       0          0                    0             0          0              0          0           0       1                 0            1                  0                  0                     0        0                   0              0                        0                  0            1                0          7                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  1                1               0             0            1               0             0         0               0            0                   0                      0                      0           1                0              0   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CA

In [48]:
df.TREATED.value_counts()

0    112540
1       417
Name: TREATED, dtype: int64

In [49]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df.to_pickle('../../matching_data/data.p')
df.shape

(112957, 78)

#### SEARCH THE IMPACT OF MILESTONES_ACHIEVED ON THE OUTCOME EFFECT

In [50]:
milestones = treated3.N_MILESTONES_ACHIEVED
milestones.value_counts().sort_index()

1      46
5       3
6       2
7       3
8       3
9       9
10     11
11      4
12      7
13      3
14      5
15      5
16     65
17    224
18     10
19      4
20      3
21      4
23      2
25      4
Name: N_MILESTONES_ACHIEVED, dtype: int64

#### Separate Treated Groups by Milestones Achieved

In [51]:
milestones.value_counts().sort_index()
outcomes = treated3[['OUTCOME', 'OUTCOME2', 'RESULT_VALUE']]

filt1 = milestones <= 16
filt2 = (milestones > 16)

one = outcomes[filt1].mean()
two = outcomes[filt2].mean()


#### Group 1

In [52]:
one

OUTCOME          0.433735
OUTCOME2         0.005752
RESULT_VALUE    33.773519
dtype: float64

#### Group 2

In [53]:
two

OUTCOME          0.442231
OUTCOME2         0.005699
RESULT_VALUE    32.985869
dtype: float64

In [54]:
minimum = outcomes.OUTCOME2 == outcomes.OUTCOME2.min()

In [55]:
treated3[minimum][['LAST_ACTIVITY_DATE','REGISTER_DATE', 'EVENT_DATE', 'OUTCOME2','MYR']]

LAST_ACTIVITY_DATE REGISTER_DATE  EVENT_DATE  OUTCOME2     MYR
123         2020-09-01    2020-06-01  2020-06-01 -0.347415  202006

In [56]:
treated_help[treated_help.PERS_ID == 3153081][['LAST_ACTIVITY_DATE','REGISTER_DATE', 'EVENT_DATE', 'OUTCOME2','MYR']]

LAST_ACTIVITY_DATE REGISTER_DATE  EVENT_DATE  OUTCOME2     MYR
516         2019-01-01    2018-07-01  2018-06-01 -0.119048  201806
517         2019-01-01    2018-07-01  2019-01-01 -0.119048  201901

In [57]:
treated_help[['LAST_ACTIVITY_DATE','REGISTER_DATE', 'EVENT_DATE','FF','RESULT_VALUE','OUTCOME', 'OUTCOME2','MYR']].head(2)

LAST_ACTIVITY_DATE REGISTER_DATE  EVENT_DATE  FF  RESULT_VALUE  OUTCOME  OUTCOME2     MYR
0         2019-08-01    2019-08-01  2019-07-01   0     27.772373        0  0.007732  201907
1         2019-08-01    2019-08-01  2019-12-01   1     27.987100        0  0.007732  201912

In [58]:
mean0  = df[df.TREATED == 0].describe().loc[['mean', 'std']]
mean1  = df[df.TREATED == 1].describe().loc[['mean', 'std']]

means = pd.concat((mean0.T, mean1.T), axis = 1)
means.columns = ['m0', 'std0', 'm1', 'std1']

#### Compare the univariate distributions on the covariates on treatment and control groups 

In [59]:
means.round(3)

m0        std0           m1        std1
AGE_AT_MIDMONTH                  55.319      17.995       50.959      12.363
ALLOW_AMT                      1265.388    4871.008      870.141    2331.387
CC_ADHD                           0.031       0.174        0.036       0.186
CC_ALZHEIMER                      0.007       0.081        0.000       0.000
CC_ALZHEIMER_PLUS                 0.022       0.147        0.000       0.000
CC_AMI                            0.004       0.066        0.002       0.049
CC_ANEMIA                         0.098       0.297        0.072       0.259
CC_ANXIETY_DISORDER               0.236       0.425        0.254       0.436
CC_ARTHRITIS                      0.217       0.412        0.180       0.385
CC_ASTHMA                         0.083       0.275        0.086       0.281
CC_ATRIAL_FIB                     0.050       0.217        0.026       0.160
CC_AUTISM                         0.004       0.066        0.002       0.049
CC_BIPOLAR                        0.037       0.189        0.031       0.174
CC_BPH                            0.045       0.208        0.019       0.137
CC_BREAST_CANCER                  0.027       0.163        0.043       0.203
CC_CATARACT                       0.110       0.313        0.079       0.270
CC_CEREBRAL_PALSY                 0.002       0.049        0.002       0.049
CC_CHRONIC_KIDNEY                 0.139       0.346        0.065       0.246
CC_COLORECTAL_CANCER              0.006       0.077        0.000       0.000
CC_COPD                           0.065       0.247        0.034       0.180
CC_CYSTIC_FIBROSIS                0.003       0.054        0.002       0.049
CC_DEPRESSION                     0.220       0.414        0.233       0.423
CC_DEPRESSIVE_DISORDERS           0.006       0.076        0.005       0.069
CC_DEVELOP_DELAYS                 0.001       0.037        0.000       0.000
CC_DIABETES                       0.190       0.393        0.036       0.186
CC_DISAB_DX_CNT                   0.112       0.367        0.082       0.315
CC_DX_CNT                         3.730       3.030        3.120       2.352
CC_ENDOMETRIAL_CANCER             0.002       0.047        0.007       0.085
CC_EPILEPSY                       0.015       0.121        0.010       0.098
CC_FIBROMYALGIA                   0.141       0.348        0.144       0.351
CC_GLAUCOMA                       0.098       0.297        0.053       0.224
CC_HEARING_IMPAIR                 0.057       0.232        0.034       0.180
CC_HEART_FAILURE                  0.057       0.232        0.026       0.160
CC_HEPATITIS_A                    0.000       0.009        0.000       0.000
CC_HEPATITIS_B_ACUTE              0.000       0.022        0.000       0.000
CC_HEPATITIS_B_CHRONIC            0.001       0.030        0.000       0.000
CC_HEPATITIS_C_ACUTE              0.000       0.019        0.000       0.000
CC_HEPATITIS_C_CHRONIC            0.005       0.069        0.000       0.000
CC_HEPATITIS_C_UNSPECIFIED        0.003       0.058        0.000       0.000
CC_HEPATITIS_D                    0.000       0.004        0.000       0.000
CC_HEPATITIS_E                    0.000       0.003        0.000       0.000
CC_HEPATITIS_GEN                  0.008       0.087        0.000       0.000
CC_HIP_FRACTURE                   0.002       0.040        0.000       0.000
CC_HYPERLIPIDEMIA                 0.376       0.484        0.254       0.436
CC_HYPERTENSION                   0.430       0.495        0.367       0.483
CC_HYPOTHYROID                    0.109       0.311        0.091       0.288
CC_INT_DISAB                      0.004       0.060        0.002       0.049
CC_ISCHEMIC                       0.161       0.368        0.070       0.255
CC_LEARN_DISAB                    0.002       0.048        0.000       0.000
CC_LEUKEMIAS                      0.010       0.098        0.012       0.109
CC_LIVER                          0.049       0.216        0.065       0.246
CC_LUNG_CANCER                   

#### Propensity score matching and distribution differences

In [60]:
columns_to_drop = ['PERS_ID', 'BIOMETRIC_CATEGORY', 'OUTCOME', 'OUTCOME2', 'RESULT_VALUE', 'TREATED']
cols = np.setdiff1d(df.columns, columns_to_drop)
causal = df[cols]
causal[['RESULT_VALUE', 'OUTCOME', 'OUTCOME2', 'TREATED']] = df[['RESULT_VALUE', 'OUTCOME', 'OUTCOME2', 'TREATED']].copy()

/usr/local/lib64/python3.6/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [61]:
from sklearn.linear_model import LogisticRegression as LG
from sklearn.neighbors import NearestNeighbors

#### Balanced Logistic Regression with L2 Regularization to get the propensities

In [62]:
model = LG(max_iter = 1000, class_weight = 'balanced').fit(causal.iloc[:,:-3].values, causal.TREATED.values)
causal['PROPENSITY'] = model.predict_proba(causal.iloc[:,:-3])[:,1]

/usr/local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### 1 by 1 matching on the treated group

In [63]:
Xtreated = causal[causal.TREATED == 1]
Xcontrols = causal[causal.TREATED == 0].reset_index(drop = True)

neighbors = NearestNeighbors(n_neighbors=1).fit(X = Xcontrols.PROPENSITY.values.reshape(-1,1))
matched_indices = neighbors.kneighbors(X = Xtreated.PROPENSITY.values.reshape(-1,1), n_neighbors=1, return_distance = False)
matched = Xcontrols.iloc[matched_indices[:,0]].reset_index(drop = True)

In [64]:
matched.head(2)

AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  CC_PRESSURE  \
0                7         99        0             0                  0       0          0                    0             0          0              0          0           0       0                 0            0                  0                  0                     0        0                   0              0                        0                  0            0                0          0                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            0                   0                      0                      0           0                0              0               0            0   
1               17       3257        0             0                  0       0          0                    0             0          1              0          0           0       0                 0            0                  0                  0                     0        0                   0              1                        0                  0            0                0          3                      0            0                0            0                  0                 0               0                     0                       0                     0                       0                           0               0               0                 0                0                  0                0               0             0            0               0             0         0               0            1                   0                      0                      0           0                0              0               0            0   

   CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  RESULT_VALUE  OUTCOME  OUTCOME2  TREATED  PROPENSITY  
0                   0        0                 0                       0               0                0          0           0                   0                 0         0         29.16        0  0.047325        0    0.529115  
1                   0        0                 0                       0               0                0          0           0                   0                 0         0         28.39        1 -0.289891        0    0.541963

In [65]:
mean0m  = matched.describe().loc[['mean', 'std']]
mean1m  = Xtreated.describe().loc[['mean', 'std']]

meansm = pd.concat((mean0m.T, mean1m.T), axis = 1)
meansm.columns = ['m0', 'std0', 'm1', 'std1']

#### Treated and Matched controls distributions comparison

In [66]:
meansm

m0         std0          m1         std1
AGE_AT_MIDMONTH              49.884892    16.823638   50.959233    12.362932
ALLOW_AMT                   822.681055  2156.413961  870.141487  2331.386903
CC_ADHD                       0.040767     0.197988    0.035971     0.186442
CC_ALZHEIMER                  0.000000     0.000000    0.000000     0.000000
CC_ALZHEIMER_PLUS             0.000000     0.000000    0.000000     0.000000
CC_AMI                        0.002398     0.048970    0.002398     0.048970
CC_ANEMIA                     0.064748     0.246377    0.071942     0.258703
CC_ANXIETY_DISORDER           0.242206     0.428933    0.254197     0.435932
CC_ARTHRITIS                  0.167866     0.374196    0.179856     0.384529
CC_ASTHMA                     0.074341     0.262639    0.086331     0.281189
CC_ATRIAL_FIB                 0.023981     0.153173    0.026379     0.160452
CC_AUTISM                     0.002398     0.048970    0.002398     0.048970
CC_BIPOLAR                    0.043165     0.203474    0.031175     0.173999
CC_BPH                        0.019185     0.137338    0.019185     0.137338
CC_BREAST_CANCER              0.023981     0.153173    0.043165     0.203474
CC_CATARACT                   0.064748     0.246377    0.079137     0.270276
CC_CEREBRAL_PALSY             0.002398     0.048970    0.002398     0.048970
CC_CHRONIC_KIDNEY             0.059952     0.237683    0.064748     0.246377
CC_COLORECTAL_CANCER          0.000000     0.000000    0.000000     0.000000
CC_COPD                       0.026379     0.160452    0.033573     0.180344
CC_CYSTIC_FIBROSIS            0.002398     0.048970    0.002398     0.048970
CC_DEPRESSION                 0.239808     0.427479    0.232614     0.423006
CC_DEPRESSIVE_DISORDERS       0.004796     0.069171    0.004796     0.069171
CC_DEVELOP_DELAYS             0.000000     0.000000    0.000000     0.000000
CC_DIABETES                   0.023981     0.153173    0.035971     0.186442
CC_DISAB_DX_CNT               0.067146     0.250576    0.081535     0.314810
CC_DX_CNT                     3.059952     2.372469    3.119904     2.352374
CC_ENDOMETRIAL_CANCER         0.004796     0.069171    0.007194     0.084615
CC_EPILEPSY                   0.004796     0.069171    0.009592     0.097587
CC_FIBROMYALGIA               0.167866     0.374196    0.143885     0.351395
CC_GLAUCOMA                   0.047962     0.213942    0.052758     0.223818
CC_HEARING_IMPAIR             0.035971     0.186442    0.033573     0.180344
CC_HEART_FAILURE              0.014388     0.119229    0.026379     0.160452
CC_HEPATITIS_A                0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_B_ACUTE          0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_B_CHRONIC        0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_C_ACUTE          0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_C_CHRONIC        0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_C_UNSPECIFIED    0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_D                0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_E                0.000000     0.000000    0.000000     0.000000
CC_HEPATITIS_GEN              0.000000     0.000000    0.000000     0.000000
CC_HIP_FRACTURE               0.000000     0.000000    0.000000     0.000000
CC_HYPERLIPIDEMIA             0.256595     0.437278    0.254197     0.435932
CC_HYPERTENSION               0.381295     0.486288    0.366906     0.482540
CC_HYPOTHYROID                0.083933     0.277620    0.091127     0.288135
CC_INT_DISAB                  0.004796     0.069171    0.002398     0.048970
CC_ISCHEMIC                   0.062350     0.242081    0.069544     0.254683
CC_LEARN_DISAB                0.000000     0.000000    0.000000     0.000000
CC_LEUKEMIAS                  0.016787     0.128625    0.011990     0.108973
CC_LIVER                      0.064748     0.246377    0.064748     0.246377
CC_LUNG_CANCER                0.